In [2]:
from __future__ import print_function, division

import pennylane as qml
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pandas as pd
import argparse
import os
import math
import sys

# from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

dev = qml.device('cirq.simulator', wires=2)

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [427]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=20, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=1, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
config = parser.parse_known_args()[0]
print(config)

Namespace(b1=0.5, b2=0.999, batch_size=1, channels=1, img_size=28, latent_dim=100, lr=0.0002, n_cpu=8, n_epochs=20, sample_interval=400)


In [428]:
# Loss function
adversarial_loss = torch.nn.BCELoss(size_average=False)
cuda = True if torch.cuda.is_available() else False

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [429]:
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 2) + \
                   np.random.normal(scale=eps, size=(3,))
gen_weights = tf.Variable(init_gen_weights)
# gen_weights = Variable(Tensor(3).fill_(np.random.normal(scale=eps, size=(1,)).item()), requires_grad=True)
gen_weights = torch.randn( 3, 1, requires_grad=True)

In [430]:
gen_weights.data

tensor([[-0.0690],
        [ 0.4344],
        [ 0.9972]])

In [431]:
disc_gen_weights = Variable(Tensor(gen_weights.data), requires_grad=True)

In [458]:
@qml.qnode(dev, interface="tf")
def gen_circuit(w):
    # Sample noise as generator input
    z = random.uniform(-1, 1)

    qml.RY(np.arcsin(z), wires=0)
    qml.RY(np.arcsin(z), wires=1)
    qml.RZ(np.arccos(z), wires=0)
    qml.RZ(np.arccos(z), wires=1)
    qml.RY(w[0], wires=0)
    qml.RY(w[1], wires=1)
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    qml.CNOT(wires=[0, 1])
    qml.RZ(w[2], wires=1)
    qml.CNOT(wires=[0, 1])

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    return qml.expval(qml.PauliZ(0))

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(1, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, sample):
        sample_flat = sample.view(sample.size(0), -1)
        validity = self.model(sample_flat)

        return validity
    
discriminator = Discriminator()
optimizer_G = torch.optim.SGD([disc_gen_weights], lr=config.lr, momentum=0.9)
# optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=config.lr, momentum=0.9)
# optimizer_G = torch.optim.Adam([gen_weights], lr=config.lr, betas=(config.b1, config.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=config.lr, betas=(config.b1, config.b2))
opt = tf.keras.optimizers.SGD(0.4)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [459]:
gen_weights

<tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([3.13738087, 0.0056535 , 0.00472762])>

In [460]:
class BimodalDataset(data.Dataset):
    """Bimodal distribution dataset."""
    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        self.df = pd.read_csv(csv_file, header=None)
    def __len__(self):
        return len(self.df.to_numpy()[0])
    def __getitem__(self, idx):
        item = self.df.to_numpy()[0][idx]
        return item 
 
filename = 'data_bimodal.csv'
dataset = BimodalDataset(filename)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=False)

In [461]:
def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

In [558]:
def generator(num_sample, gen_weights):
#     w = list(gen_weights.detach().numpy())
#     w = gen_weights.numpy()
    w = [gen_weights.detach().numpy()[i].item() for i in range(3)]
    
    samples = []
    for i in range(num_sample):
        samples.append(gen_circuit(w))
    return Tensor(samples)
generator(10, gen_weights)

TypeError: Scalar tensor has no `len()`

In [556]:
gen_weights = Variable(Tensor(3).fill_(np.random.normal(scale=eps, size=(1,)).item()), requires_grad=True)
gen_weights = torch.randn( 3, 1, requires_grad=True)
# gen_samples = Variable(Tensor(generator(10, gen_weights)), requires_grad=False)
# gen_samples
gen_weights

tensor([[1.1708],
        [0.1696],
        [0.1256]], requires_grad=True)

In [579]:
gen_weights

<tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([3.14701869e+00, 4.62267226e-03, 1.32276378e-03])>

In [583]:
# Adversarial ground truths
valid = Variable(Tensor(config.batch_size).fill_(1.0), requires_grad=False)
fake = Variable(Tensor(config.batch_size).fill_(0.0), requires_grad=False)
# gen_weights = Variable(Tensor(3).fill_(np.random.normal(scale=eps, size=(1,)).item()), requires_grad=True)
# gen_weights = torch.randn( 3, 1, requires_grad=True)

init_gen_weights = np.array([np.pi] + [0] * 2) + \
                   np.random.normal(scale=eps, size=(3,))
gen_weights = tf.Variable(init_gen_weights)
# optimizer_G = torch.optim.Adam([gen_weights], lr=config.lr)#, momentum=0.9

for epoch in range(1):
    for i, samples in enumerate(dataloader):
        # Configure input
        real_samples = Variable(samples.type(Tensor))
        
        # -----------------
        #  Train Generator
        # -----------------
        
#         #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#         optimizer_G.zero_grad()
        
#         # Generate a batch of images
#         # TODO: gen_weights is never updated
#         # Two possible ways for solving this issue: check adam optimizer, or use tf.keras
# #         gen_samples = Tensor([generator(gen_weights.detach().numpy()) for i in range(len(samples))])
# #         gen_samples = generator(len(samples), gen_weights)
        
#         gen_samples = generator(config.batch_size, gen_weights)
#         # Loss measures generator's ability to fool the discriminator
#         g_loss = adversarial_loss(discriminator(gen_samples), valid) #.view(config.batch_size)
#         print(gen_weights)
        
#         g_loss.backward()
#         optimizer_G.step()
#         #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Using keras for updating generator parameters
        def gen_cost(gen_weights):
        #     w = list(gen_weights.detach().numpy())
        #     w = gen_weights.numpy()

            gen_samples = []
            for i in range(config.batch_size):
                gen_samples.append(gen_circuit(gen_weights))
            gen_samples = Tensor([gen_samples[i].numpy() for i in range(len(gen_samples))])
#             print('gen samples', gen_samples)
            loss = adversarial_loss(discriminator(gen_samples), valid)
            loss = tf.constant(loss.detach().numpy())
            return loss
        
        print(gen_cost(gen_weights))
        cost = lambda: gen_cost(gen_weights)
        opt.minimize(cost, gen_weights)
        cost_val = cost().numpy()
        print("Cost = {}".format(cost_val))
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        
        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_samples), valid)
        fake_loss = adversarial_loss(discriminator(gen_samples.view(config.batch_size)), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
#         for j in discriminator.parameters():
#             print(j[0])

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, config.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )
        if i == 1:
            break

tf.Tensor(0.70248663, shape=(), dtype=float32)


ValueError: No gradients provided for any variable: ['Variable:0'].

In [576]:
def gen_cost(gen_weights):
#     w = list(gen_weights.detach().numpy())
#     w = gen_weights.numpy()

    gen_samples = []
    for i in range(2):
        gen_samples.append(gen_circuit(gen_weights))
    gen_samples = [gen_samples[i] for i in range(len(gen_samples))]
    loss = adversarial_loss(discriminator(Tensor(gen_samples)), valid)
    loss = tf.constant(loss.detach().numpy())
    return gen_circuit(gen_weights)

In [577]:
gen_cost(gen_weights)

TypeError: Scalar tensor has no `len()`

In [535]:
discriminator(gen_samples)

tensor([[0.4944]], grad_fn=<SigmoidBackward>)

In [565]:
bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true, y_pred).numpy()

tensor([[-2.6544,  0.8146],
        [-0.6844,  0.5261],
        [ 0.9078,  1.0360]], requires_grad=True)

In [379]:
torch.randn( 2, 1, requires_grad=True)

tensor([[-0.1289],
        [-0.7562]], requires_grad=True)

In [364]:
import torch
import torch.nn as nn
# torch.autograd.Variable is roughly equivalent to requires_grad=True
# and is deprecated in PyTorch 1.0

# your code gives not reason to have `requires_grad=True` on `x_data`
x_data = torch.tensor( [ [1.0, 2.0], [2.0, 3.0], [3.0, 4.0] ])
y_data = torch.tensor( [ [2.0], [4.0], [6.0] ] )

w = torch.randn( 2, 1, requires_grad=True)
# b = torch.randn( 1, 1, requires_grad=True)

def model(x2, w2):
#     print(x2)
#     print(w2)
#     print(x2 @ w2)
    return x2 @ w

criterion = torch.nn.MSELoss( size_average=False)
optimizer = torch.optim.SGD([w], lr=0.01)

for epoch in range(2) :
    y_pred = model(x_data, w)
    loss = criterion( y_pred, y_data)
    print( epoch, loss.data.item())
    print(loss)
    print(w)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 57.53794479370117
tensor(57.5379, grad_fn=<MseLossBackward>)
tensor([[-1.0662],
        [ 0.7301]], requires_grad=True)
1 2.3487346172332764
tensor(2.3487, grad_fn=<MseLossBackward>)
tensor([[-0.4997],
        [ 1.5331]], requires_grad=True)


In [346]:
x_data

tensor([[1., 2.],
        [2., 3.],
        [3., 4.]])

In [375]:
class Generator():
    def __init__(self, num_sample):
        self.params = nn.Linear(3, 1).weight
    def get_samples

In [374]:
import torch
import torch.nn as nn

x_data = torch.tensor( [ [1.0, 2.0], [2.0, 3.0], [3.0, 4.0] ] )
y_data = torch.tensor( [ [2.0], [4.0], [6.0] ] )

model = nn.Linear(2, 1)
print(model.weight)

criterion = torch.nn.MSELoss( size_average=False )
optimizer = torch.optim.SGD([model.weight], lr=0.01 )

for epoch in range(10) :
    y_pred = model(x_data)
    loss = criterion( y_pred, y_data )
    print( epoch, loss.data.item() )
    print(model.weight)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Parameter containing:
tensor([[-0.0536,  0.0364]], requires_grad=True)
0 50.25956726074219
Parameter containing:
tensor([[-0.0536,  0.0364]], requires_grad=True)
1 1.602331280708313
Parameter containing:
tensor([[0.4767, 0.7915]], requires_grad=True)
2 0.6068929433822632
Parameter containing:
tensor([[0.5565, 0.8966]], requires_grad=True)
3 0.5833829045295715
Parameter containing:
tensor([[0.5719, 0.9088]], requires_grad=True)
4 0.5797111988067627
Parameter containing:
tensor([[0.5782, 0.9077]], requires_grad=True)
5 0.5764609575271606
Parameter containing:
tensor([[0.5831, 0.9048]], requires_grad=True)
6 0.5732367634773254
Parameter containing:
tensor([[0.5878, 0.9016]], requires_grad=True)
7 0.5700318217277527
Parameter containing:
tensor([[0.5924, 0.8984]], requires_grad=True)
8 0.5668442249298096
Parameter containing:
tensor([[0.5971, 0.8952]], requires_grad=True)
9 0.5636747479438782
Parameter containing:
tensor([[0.6017, 0.8920]], requires_grad=True)


In [377]:
nn.Linear(3, 1).weight

Parameter containing:
tensor([[-0.2477, -0.1780,  0.0196]], requires_grad=True)

In [378]:
model.parameters()

<generator object Module.parameters at 0x7f36ab45a048>